In [1]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

# Coherence Map

In [2]:
import equinox as eqx
import jax.numpy as jnp
import numpy as np

from differt.geometry.utils import path_lengths
from differt.plotting import draw_image, reuse, set_defaults
from differt.scene.sionna import download_sionna_scenes, get_sionna_scene
from differt.scene.triangle_scene import TriangleScene

In [3]:
download_sionna_scenes()  # Let's download Sionna scenes (from the main branch)

In [6]:
set_defaults("plotly")

file = get_sionna_scene("simple_street_canyon")
scene = TriangleScene.load_xml(file)

scene = eqx.tree_at(lambda s: s.transmitters, scene, jnp.array([-33.0, 0.0, 32.0]))
batch = (50, 50)
scene_grid = scene.with_receivers_grid(*batch)
power = jnp.zeros(batch)
groups = jnp.zeros(batch, dtype=jnp.int32)

x, y, z = jnp.unstack(scene_grid.receivers, axis=-1)

with reuse() as fig:
    scene.plot()

    for order in range(2):
        for paths in scene_grid.compute_paths(order=order, chunk_size=1_000):
            new_groups = jnp.unique(
                paths.mask.reshape(-1, paths.mask.shape[-1]), axis=0, return_inverse=True
            )[1].reshape(paths.mask.shape[:-1])

            groups = jnp.unique(
                jnp.stack((groups, new_groups), axis=-1).reshape(-1, 2), axis=0, return_inverse=True
            )[1].reshape(batch)

            power += (paths.mask / path_lengths(paths.vertices) ** 2).sum(axis=-1)

    draw_image(
        np.asarray(power), x=np.asarray(x), y=np.asarray(y), z0=float(z.ravel()[0])
    )

fig

triangle_vertices.shape =(1, 0, 3, 3), paths.shape = (1, 2500, 1, 0, 3)
intersect.shape = (1, 2500, 1)
triangle_vertices.shape =(74, 1, 3, 3), paths.shape = (1, 2500, 74, 1, 3)
intersect.shape = (1, 2500, 74)


In [7]:
with reuse() as fig:
    scene.plot()
    draw_image(
        np.asarray(groups / groups.max()), x=np.asarray(x), y=np.asarray(y), z0=float(z.ravel()[0])
    )

fig